# Task 1: API ที่นำมาใช้

จัดทำโดย นายตุลธร วงศ์ชัย รหัสนักศึกษา 63070224

สำหรับ API ที่นำมาใช้งานมีชื่อว่า ภาษีไปไหน? ซึ่งเกี่ยวกับการใช้งบประมาณจากภาษีของประชาชนแจกจ่ายไปให้แก่หน่วยงานของภาครัฐ source: https://govspending.data.go.th/api/documentation

## Import Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import urllib.request
import json
import csv
from datetime import datetime
import numpy as np

In [2]:
api_key = "Y8DPWYVlNC4668nI7DjJS3GSmQR4ZcVR"

# Task 2: Collect data from API

ทางผู้จัดทำต้องการข้อมูลของหน่วยงานที่อยู่ใน `กรุงเทพมหานคร` จึงมีวิธีขึ้นตอนดังนี้
1. ทำการ requests เพื่อหา `dept_code` ของ `กรุงเทพมหานคร`
2. นำ `dept_code` ที่ได้จากขึ้นตอนแรกมาเป็น parameter เพื่อใช้ในการ requests ข้อมูลของ `กรุงเทพมหานคร`
3. ทำการเขียนข้อมูลที่ได้รับ response กลับมาให้อยู่ใน format `.csv`

In [3]:
def find_dept_code(dept_name, api_key):
  url = f"https://opend.data.go.th/govspending/egpdepartment?api-key={api_key}&limit=10&dept_name={dept_name}"
  response = requests.get(url)
  data = response.json()

  bkk_code = ''
  for i in data['result']:
    if i['dept_name'] == 'กรุงเทพมหานคร':
      bkk_code = i['dept_code']
      break

  return bkk_code  # string

def get_raw_data(limit, api_key, year, dept_code):
  url = f"https://opend.data.go.th/govspending/cgdcontract?api-key={api_key}&year={year}&limit={limit}&dept_code={dept_code}"
  response = requests.get(url)
  data = response.json()
  data = data['result']

  for i in data: # ทำการแตกข้อมูลใน list ของ contract ออกมา
    i.update(i['contract'][0])
    del i['contract']

  return data # object(dict)

def write_to_csv(file_path, heading, rows):
  with open(file_path, 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=heading)
    writer.writeheader()
    writer.writerows(rows)


# Task 3: Parse the data to .csv format

สร้าง folder ที่ชื่อว่า data แล้วสร้างไฟล์เปล่า bkk_CGDContract_data.csv ด้วย UNIX Command

In [4]:
!mkdir data
!cd data && touch bkk_CGDContract_data.csv

'touch' is not recognized as an internal or external command,
operable program or batch file.


นำ function ที่สร้างจาก Task2 มาใช้ในการดึงข้อมูล โดยจะดึงข้อมูลจากหน่วยงาน `กรุงเทพมหานคร` จากปี พ.ศ. 2558 - 2564 ปีละ 400 objects รวมเป็น 2800 objects

In [5]:
# หา dept_code ของ "กรุงเทพมหานคร"
bkk_dept_code = find_dept_code("กรุงเทพมหานคร", api_key)

# ดึงข้อมูลจากหน่วยงาน "กรุงเทพมหานคร" จากปี พ.ศ. 2558 - 2564 ปีละ 400 objects รวมเป็น 2800 objects
raw_data = []
for year in range(2558, 2565):
  raw_data += get_raw_data(400, api_key, year, bkk_dept_code)

ChunkedEncodingError: ('Connection broken: IncompleteRead(3965 bytes read, 4227 more expected)', IncompleteRead(3965 bytes read, 4227 more expected))

In [ ]:
# หา columns ของข้อมูล
heading = list(raw_data[0].keys())
heading

เขียนข้อมูลลงบนไฟล์ bkk_CGDContract_data.csv ไว้ใน folder data

In [ ]:
write_to_csv('data/bkk_CGDContract_data.csv', heading, raw_data)

# Task 4: Load Data and Pre-processing

หลังจากได้ไฟล์ bkk_CGDContract_data.csv ที่มีข้อมูลทั้งหมด 2800 แถว จึงนำมาทำการอ่านข้อมูลใน pandas และทำการ pre-processing ก่อนนำไปวิเคราะห์ต่อไป

In [ ]:
df = pd.read_csv('data/bkk_CGDContract_data.csv', encoding='utf8')
df

Replace "-" to null

In [ ]:
df = df.replace({'-': None})

 เนื่องจากข้อมูลวันที่อยู่ในรูปแบบของไทย จึงสร้างฟังชั่นเพื่อแปลงให้อยู่ในรูปแบบที่อ่านได้ใน pandas
 ได้แก่ columns `announce_date`, `transaction_date`, `contract_date`, `contract_finish_date`

In [ ]:
# Parse Thai date to general date
def convert_thai_date(value):
  if value == None:
    return None

  value = value.split(' ')
  day, th_month, th_year = value[0], value[1], '25' + value[2]

  thai_abbr_months = {
      "ม.ค.": "01",
      "ก.พ.": "02",
      "มี.ค.": "03",
      "เม.ย.": "04",
      "พ.ค.": "05",
      "มิ.ย.": "06",
      "ก.ค.": "07",
      "ส.ค.": "08",
      "ก.ย.": "09",
      "ต.ค.": "10",
      "พ.ย.": "11",
      "ธ.ค.": "12",
  }

  # แปลงให้อยู่ในรูปแบบ str -> 'y/m/d' แล้วนำมาแปลงเป็นข้อมูลวันที่ด้วย packages datetime.strptime
  converted_date = str(int(th_year) - 543) + "/" + thai_abbr_months[th_month] + "/" + str(day)
  date_obj = datetime.strptime(converted_date , '%Y/%m/%d').date()

  return date_obj

date_cols = ['announce_date', 'transaction_date', 'contract_date', 'contract_finish_date']

for col in date_cols:
  df[col] = df[col].apply(lambda x: convert_thai_date(x))

เนื่องจากข้อมูลตัวเลขอยู่ในรูปแบบ string และคั่นหลักด้วย "," จึงทำการแปลงให้เป็น integer
ได้แก่ columns `project_money`, `price_build`, `sum_price_agree`, `price_agree`

In [ ]:
# Parse comma number to integer
price_cols = ['project_money', 'price_build', 'sum_price_agree', 'price_agree']

for col in price_cols:
  df[col] = df[col].apply(lambda x: int(str(x).replace(',', '')) if x != None else x)

ทำ budget_year ให้เป็น ค.ศ.

In [ ]:
df['budget_year'] = df['budget_year'] - 543

Drop columns ที่ไม่จะไม่นำมาใช้
    ** province และ dept_name มีค่าเดียวคือ `กรุงเทพมหานคร`

In [ ]:
df = df.drop(columns=['project_id', 'province', 'dept_name', 'project_location', 'geom', 'winner_tin', 'contract_no'])

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

ตรวจสอบค่า missing ของ `district` และ `sub_district` แล้วพบว่า `dept_sub_name` มีค่าเป็น `สำนักงานเขตบางนา` ทั้งหมด จึง fill ค่า missing ด้วย `บางนา`

In [ ]:
# Check missing of district
df[df['district'].isnull()]

In [ ]:
df = df.fillna({"district":"บางนา", "subdistrict":"บางนา"})

ตรวจสอบค่า missing ของ `winner` แล้วพบว่า `project_name` มีค่าเป็น `เช่าที่ดินจากสำนักงานทรัพย์สินส่วนพระมหากษัตริย์` ซึ่งเกี่ยวข้องกัน จึง fill ค่า missing ด้วย `สำนักงานทรัพย์สินส่วนพระมหากษัตริย์`

In [ ]:
# Check missing of winner
df[df['winner'].isnull()]

In [ ]:
df = df.fillna({"winner":"สำนักงานทรัพย์สินส่วนพระมหากษัตริย์"})

ทำการ fill missing ของ `price_build` (ราคากลาง) ด้วยค่าเฉลี่ยของ `project_money` (งบประมาณ) และ `sum_price_agree` (ราคาที่ตกลง)

In [ ]:
df[df['price_build'].isnull()]

In [ ]:
fill_price_build = (df['project_money'] + df['sum_price_agree']) / 2
df = df.fillna({"price_build": fill_price_build})

`announce_date` และ `contract_finish_date` ไม่มีวิธีการที่จะสามารถ fill missing ได้ จึงละเว้นให้เป็นค่า missing

In [ ]:
# announce_date and contract_finish_date can't be fill
df.isnull().sum()

# Task 5: Analyse and Summarise

หลังจากที่นำข้อมูลมาทำการ pre-processing แล้วจึงมาทำการวิเคราะห์ต่อ

In [ ]:
# Set Thai font for matplotlib visualiztion
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['SF Thonburi']

In [ ]:
df.describe()

ทำการแสดง bar chart ของงบประมาณรวมทั้งหมดในแต่ละปี
- การใช้งบประมาณมีแนวโน้มสูงขึ้นตั้งแต่ปี 2018

In [ ]:
# Bar char of budget money between 2015-2021
df.groupby('budget_year')['project_money'].sum().plot(kind='barh')
plt.title("จำนวนงบประมาณในปี 2015 - 2021", weight='bold', fontsize=14)
plt.xlabel('หมื่นล้านบาท')
plt.show()

แสดง line plot ของจำนวนงบประมาณในแต่ละปี แบ่งตามประเภทของโครงการ
- พบว่าโครงการประเภทก่อสร้างมีแนวโน้มที่ใช้งบประมาณที่สูงขึ้น

In [ ]:
# line plot of budget money between 2015-2021 by project type
df.groupby(['budget_year', 'project_type_name'])['project_money'].sum().unstack().plot(figsize=(10,5), style='.-')
plt.title("จำนวนงบประมาณในปี 2015 - 2021 แยกตามประเภทโครงการ", weight='bold', fontsize=14)
plt.ylabel('หมื่นล้านบาท')
plt.grid(color = 'grey', linestyle = '--', linewidth = 0.5)
plt.show()

In [ ]:
top_10_type = pd.DataFrame(df.groupby(['project_type_name'])['project_money'].sum())
top_10_type

แสดง scatter plot ของการกระจายงบประมาณเทียบกับวันที่ทำสัญญา
- ตั้งแต่ปี 2018 มีการกระจายที่มากขึ้น แสดงถึงการใช้งบประมาณที่สูงขึ้น

In [ ]:
plt.figure(figsize=(30,10))
plt.scatter(df['contract_date'], df['project_money'], color='red')
plt.ylabel('หมี่นล้านบาท')
plt.title("การกระจายของงบประมาณนับตั้งแต่วันทำสัญญาโครงการ", weight='bold', fontsize=14)
plt.show()

แสดง pie chart ของผู้ชนะในการประมูลสัญญาจ้างมากสุด 5 อันดับ
- `บริษัท ปตท. จำกัด(มหาชน)` ชนะไปถึง 385 ครั้ง
- และเป็นการ `ซื้อ` ทั้งหมด คาดว่าเป็นการซื้อน้ำมันจาก `บริษัท ปตท. จำกัด(มหาชน)`

In [ ]:
# top 10 winner

top_winner = df.groupby('winner')['winner'].count().sort_values(ascending=False).head(5)

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(aspect="equal"), facecolor='white')
key = top_winner.keys()
data = top_winner

def func(pct, allvals):
    absolute = int(np.round(pct/100.*np.sum(allvals)))
    return "{:.1f}%\n({:d} ครั้ง)".format(pct, absolute)

wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data),
                                  textprops=dict(color="w"))
ax.legend(wedges, key,
          title="Winners",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))

plt.setp(autotexts, size=11, weight="bold")
ax.set_title("ผู้ชนะในการประมูลสัญญาจ้างมากสุด 5 อันดับ", fontsize=16, weight="bold")


plt.show()

In [ ]:
df[df['winner'] == 'บริษัท ปตท. จำกัด(มหาชน)'].groupby('project_type_name')['project_type_name'].count()

แสดง 10 อันดับหน่วยงานที่ใช้งบประมาณรวมสูงสุด

In [ ]:
top_10_dept = pd.DataFrame(df.groupby('dept_sub_name')['project_money'].sum())
top_10_dept.nlargest(n=10,columns=['project_money'])

แสดง 10 อันดับเขตที่ใช้งบประมาณรวมสูงสุด

In [ ]:
top_10_dist = pd.DataFrame(df.groupby(['district'])['project_money'].sum())
top_10_dist.nlargest(n=10,columns=['project_money'])

แสดงอัตราส่วนของสถานะโครงการที่ `อยู่ระหว่างดำเนินการ` และ `สิ้นสุดสัญญา`


In [ ]:
# plot project status
status = df.groupby(['budget_year', 'project_status']).size().unstack(fill_value=0)

ax = status.plot.bar(color=['orange', 'green'], figsize=(10,5))
plt.ylabel("จำนวน")
plt.title("สถานะสัญญาในปีงบประมาณ 2015 - 2021", weight='bold', fontsize=14)
plt.legend(loc="lower left",bbox_to_anchor=(0.8,1.0))
for container in ax.containers:
    ax.bar_label(container)

plt.show()

แสดง line plot และ scatter plot ของราคาสัญญาที่ตกลงเปรียบเทียบกับเส้นราคากลาง
- พบว่างบประมาณในช่วงราคาที่ยังไม่สูงยังไม่ห่างจากเส้นราคากลางมาก
- แต่ในช่วงงบประมาณที่สูงขึ้นจะสามารถตกลงราคาที่ต่ำกว่าราคากลางได้

In [ ]:
# plot price_agree compare to price_build
plt.figure(figsize=(8,5))
plt.scatter(df['project_money'], df['price_agree'], color='green')
plt.plot(df['price_build'],df['price_build'], color='red')
plt.xlabel('project_money (หมื่นล้านบาท)')
plt.ylabel('price_agree (หมื่นล้านบาท)')
plt.title("ราคาสัญญาที่ตกลงเปรียบเทียบกับเส้นราคากลาง", weight='bold', fontsize=14)
plt.show()

## Summary

- งบประมาณใน `กรุงเทพมหานคร` มีแนวโน้มที่สูงขึ้นตั้งแต่ปี 2018 ซึ่งโครงการที่ใช้งบประมาณมากที่สุดจะเกี่ยวข้องกับ `การก่อสร้าง`
- `บริษัท ปตท. จำกัด(มหาชน)` เป็นผู้ที่ชนะการประมูลโครงการได้มากที่สุด ที่ 385 ครั้ง เป็นโครงการการซื้อน้ำมันทั้งหมด
- `ดินแดง` เป็นเขตที่ใช้งบประมาณสูงที่สุดและหน่วยงาน `กองกำจัดมูลฝอย สำนักสิ่งแวดล้อม` เป็นหน่วยงานที่ใช้งบประมาณสูงที่สุด
- งบประมาณยิ่งสูง ยิ่งสามารถตกลงได้ต่ำกว่าราคากลาง

ปัญหาที่พบ
- เป็นข้อมูลที่เกี่ยวกับโครงการสัญญาของภาครัฐ ทำให้ผู้จัดทำที่ยังไม่มีประสบการณ์ไม่สามารถเข้าใจระเบียบวิธีการของสัญญาที่ซับซ้อนได้มากนัก
- ช้อมูลส่วนมากเป็นข้อมูล categorical